In [ ]:

#Test comment

In [ ]:
import os
import pandas as pd
import  numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
import ipywidgets as widgets
from ipywidgets import interact
import plotly.express as px
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
import base64
import io
#import pandas.io.formats.style
warnings.filterwarnings('ignore')
# %matplotlib inline


In [ ]:
# Get filepath from the back end
file_path = 'FILE_PATH_PLACEHOLDER'

In [ ]:
data = pd.read_csv(file_path)

#data.head()

<h1 align='center'>Standards Report</h1>

In [ ]:
def export_report(df):
    # Custom CSS for table layout and responsiveness
    css = """
    <style>
    /* General table styling */
    table {
        width: 100%;
        border-collapse: collapse;
        table-layout: auto;
        max-width: 100%;
    }
    th, td {
        padding: 8px;
        text-align: left;
        border: 1px solid #ddd;
        word-wrap: break-word;
    }
    
    /* Responsive table container for small screens */
    .scrollable-table {
        display: block;
        width: 100%;
        overflow-x: auto;
        max-width: 100%;
        white-space: nowrap;
    }

    /* Responsive adjustments for small screens */
    @media only screen and (max-width: 768px) {
        th, td {
            min-width: 120px;
            font-size: 12px;
        }
    }
    
    /* Print-specific styles */
    @media print {
        .navbar, .footer, .sidebar, .button { 
            display: none; 
        }

        .scrollable-table {
            overflow: visible;
        }
        
        body {
            margin: 0;
            padding: 0;
        }
        
        table {
            page-break-inside: auto;
            width: 100%;
        }
        
        tr {
            page-break-inside: avoid;
            page-break-after: auto;
        }
    }
    </style>
    """
    
    
    # Otherwise, convert the regular DataFrame to HTML
    df_html = f'<div class="scrollable-table">{df.to_html()}</div>'
    
    # Combine CSS and DataFrame HTML
    full_html = css + df_html
    display(HTML(full_html))
    

In [ ]:
def export_report_styled(df_or_styler):
    css = """
    <style>
    table {
        width: 100%;
        border-collapse: collapse;
        table-layout: auto;
        max-width: 100%;
    }
    th, td {
        padding: 8px;
        text-align: left;
        border: 1px solid #ddd;
        word-wrap: break-word;
    }
    .scrollable-table {
        display: block;
        width: 100%;
        overflow-x: auto;
        max-width: 100%;
        white-space: nowrap;
    }
    @media only screen and (max-width: 768px) {
        th, td {
            min-width: 120px;
            font-size: 12px;
        }
    }
    @media print {
        .scrollable-table {
            overflow: visible;
        }
        body {
            margin: 0;
            padding: 0;
        }
        table {
            page-break-inside: auto;
            width: 100%;
        }
        tr {
            page-break-inside: avoid;
            page-break-after: auto;
        }
    }
    </style>
    """

    # Check if the input is a Styler object
    if isinstance(df_or_styler, pd.io.formats.style.Styler):
        # Use to_html() on Styler to retain styles
        df_html = f'<div class="scrollable-table">{df_or_styler.to_html()}</div>'
    else:
        # Otherwise, convert the regular DataFrame to HTML
        df_html = f'<div class="scrollable-table">{df_or_styler.to_html()}</div>'
    
    # Combine CSS and DataFrame HTML
    full_html = css + df_html
    display(HTML(full_html))



In [ ]:
data_rounded = data.copy()
numeric_columns = data.select_dtypes(include='number').columns
data_rounded[numeric_columns].round(2)
styled_data = data_rounded.style.set_properties(**{'font-size': '8pt'}).set_table_styles([{'selector': 'th', 'props': [{'font-size', '8pt'}]}]).format({col: "{:.2f}" for col in numeric_columns}).hide(axis='index')
export_report(styled_data)

In [ ]:
# Replace some cells with 'No data' with Numpy NaN
for col in data.columns:
   data[col] = data[col].replace('No data', np.nan)

In [ ]:
#data.iloc[:, 9:].dtypes

In [ ]:
# Transform data to float so we don't get errors later on 
data.iloc[:, 9:] = data.iloc[:, 9:].astype(float)
for column in data.iloc[:, 9:]:
    if data[column].dtype == 'object':
        data[column] = data[column].astype(np.float64)

In [ ]:
#data.iloc[:, 9:].dtypes

<h1 align="center">Statistics</h1>

In [ ]:
def statistics(minerals):
  min_dict = {}
  for mineral in data[minerals]:
    count = data[mineral].count()
    std = data[mineral].std()
    max = data[mineral].max()
    min = data[mineral].min()
    mean = data[mineral].mean()
    mean_plus_10 = mean + (mean * 10/100)
    mean_minus_10 = mean - (mean * 10/100)
    ok_at_10 = np.sum(data[mineral] < mean_plus_10) - np.sum(data[mineral] <= mean_minus_10)
    failure_greater_10 = count - ok_at_10
    percent_failure_greater_10 = math.ceil((failure_greater_10 / count) * 100)
    mean_plus_minus_10 = f"{mean.round(2)} ± {((10/100)*mean).round(2)}"
    mean_plus_2SD = mean + (std * 2)
    mean_minus_2SD = mean - (std * 2)
    ok_at_2SD = np.sum(data[mineral] <= mean_plus_2SD) - np.sum(data[mineral] < mean_minus_2SD)
    failure_greater_2SD = count - ok_at_2SD
    percent_failure_greater_2SD = math.ceil((failure_greater_2SD / count) * 100)
    mean_plus_minus_2SD = f"{mean.round(2)} ± {((2*std)/mean).round(2)}"
    mean_plus_3SD = mean + (std * 3)
    mean_minus_3SD = mean - (std * 3)
    ok_at_3SD = np.sum(data[mineral] <= mean_plus_3SD) - np.sum(data[mineral] < mean_minus_3SD)
    failure_greater_3SD = count - ok_at_3SD
    percent_failure_greater_3SD = math.ceil((failure_greater_3SD / count) * 100)
    mean_plus_minus_3SD = f"{mean.round(2)} ± {((3*std)/mean).round(2)}"
    RSD = (std / mean) * 100
    RSD = round(RSD, 2)
    certification = 'Certified' if RSD <= 5 else 'Provisional' if RSD > 5 and RSD <= 15 else  'Informational'
    acceptable_qc = 'Pass' if percent_failure_greater_10 <= 10 and RSD <= 15 else 'Fail'
    TwoSD_less_than_10 = '2SD > 10%' if (2 * std) > (0.1 * mean)  else 'yes'

    #print(mineral, 'count:', count, 'std:', std, 'max:', max, 'min:', min, 'mean:', mean, 'mean + 10%:', mean_plus_10, 'mean - 10%: ', mean_minus_10, 'mean + 2D: ', mean_plus_2SD, 'mean - 2D: ', mean_minus_2SD)
    min_dict[mineral] = [count, std, max, min, mean, mean_plus_10, mean_minus_10, ok_at_10, failure_greater_10, percent_failure_greater_10, mean_plus_minus_10, mean_plus_2SD, mean_minus_2SD, ok_at_2SD, failure_greater_2SD, percent_failure_greater_2SD, mean_plus_minus_2SD, mean_plus_3SD, mean_minus_3SD,
                         ok_at_3SD, failure_greater_3SD, percent_failure_greater_3SD, mean_minus_3SD, RSD, certification, acceptable_qc, TwoSD_less_than_10]
  return min_dict


In [ ]:
stat_data = list(data.columns[9:].values.tolist())
stats = statistics(stat_data)
stats_df = pd.DataFrame.from_dict(stats, orient='index', columns=['count', 'std', 'max', 'min', 'mean', 'mean + 10%', 'mean - 10%', 'Ok @ 10', 'Failures > 10%', '% Failures > 10%', 'Mean ± 10%', 'mean + 2SD', 'mean - 2SD', 'Ok @ 2SD', 'Failures > 2SD', '% Failures > 2SD', 'Mean ± 2SD', 'mean + 3SD', 'mean - 3SD', 'Ok @ 3SD', 'Failures > 3SD', '% Failures > 3SD', 'Mean ± 3SD',
                                                                  'RSD', 'Certification' , 'Acceptable QC?', '2SD < 10%'])
#stats_df

In [ ]:
def style_cert(s):
  return [
      'background-color: red' if v == 'Informational' else
      'background-color: yellow' if v == 'Provisional' else
      'background-color: green' for v in s
  ]
def style_qc(s):
  return [
      'color: blue' if v == 'Pass' else
      'color: red' for v in s
  ]
def style_2SD(s):
  return [
      'color: red' if v == '2SD > 10%' else
      '' for v in s
  ]
df_rounded = stats_df.copy()
numeric_columns = data.select_dtypes(include='number').columns
df_rounded[numeric_columns] = data[numeric_columns].round(2)
styled_df = df_rounded.style.apply(style_cert, subset=['Certification']).apply(style_qc, subset=['Acceptable QC?']).apply(style_2SD, subset=['2SD < 10%']).set_properties(**{'font-size': '8pt'}).set_table_styles([{'selector': 'th', 'props': [('font-size', '8pt')]}]).format({col: "{:.2f}" for col in numeric_columns}).hide(axis='index')
export_report_styled(styled_df)


In [ ]:
# Function to create a downloadable Excel file with two sheets
def create_download_link_multi_sheet(df1, df2, filename="Report.xlsx"):
    # Save the DataFrames to an in-memory Excel file with two sheets
    excel_buffer = io.BytesIO()
    with pd.ExcelWriter(excel_buffer, engine='openpyxl') as writer:
        df1.to_excel(writer, sheet_name='Sheet1', index=False)
        df2.to_excel(writer, sheet_name='Sheet2', index=False)
    
    excel_buffer.seek(0)  # Go to the beginning of the in-memory file
    
    # Convert the Excel file to a base64-encoded string
    b64 = base64.b64encode(excel_buffer.read()).decode()
    
    # Create the HTML button with the download link
    return HTML(f"""
        <a download="{filename}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" target="_blank">
            <button style="font-size:16px; padding:10px; color:white; background-color:green; border:none; border-radius:5px;">
                Download Excel
            </button>
        </a>
    """)

# Display the button
create_download_link_multi_sheet(styled_data, styled_df)

<h1 align="center">Plots</h1>

In [ ]:

# Plotting function
def plot_stats_sequence(minerals):
    plot_stats_sequence_fig = go.Figure()
    x_data = data['Seq#']
    for mineral in minerals: 
    
        # Handling the data (scalars vs arrays)
        y_data_mean = [stats_df['mean'][mineral]] * len(x_data) if isinstance(stats_df['mean'][mineral], (np.float64, float)) else stats_df['mean'][mineral]
        y_data_plus_10 = [stats_df[f'mean + 10%'][mineral]] * len(x_data) if isinstance(stats_df[f'mean + 10%'][mineral], (np.float64, float)) else stats_df[f'mean + 10%'][mineral]
        y_data_minus_10 = [stats_df[f'mean - 10%'][mineral]] * len(x_data) if isinstance(stats_df[f'mean - 10%'][mineral], (np.float64, float)) else stats_df[f'mean - 10%'][mineral]
        y_data_analysis = data[mineral]

        # Create the plot
        #fig = go.Figure()

        plot_stats_sequence_fig.add_trace(go.Scatter(x=x_data, y=y_data_mean, mode='lines', name='mean', visible=False))
        plot_stats_sequence_fig.add_trace(go.Scatter(x=x_data, y=y_data_analysis, mode='markers', name='Analysis', visible=False, marker=dict(color='red')))
        plot_stats_sequence_fig.add_trace(go.Scatter(x=x_data, y=y_data_analysis, mode='lines', visible=False, line=dict(color='red')))
        plot_stats_sequence_fig.add_trace(go.Scatter(x=x_data, y=y_data_plus_10, mode='lines', name='mean + 10%', visible=False, line=dict(dash='dash', color='red')))
        plot_stats_sequence_fig.add_trace(go.Scatter(x=x_data, y=y_data_minus_10, mode='lines', name='mean - 10%', visible=False, line=dict(dash='dash', color='red')))
    
    for i in  range(5): 
        plot_stats_sequence_fig.data[i].visible = True

    dropdown_buttons = []

    for i, mineral in enumerate(minerals):
        dropdown_buttons.append({
            'args': [{'visible': [False] * len(plot_stats_sequence_fig.data)},
                     {'title': f"Evelyn Project {data['Original QC'][0]} {data['Standard'][0]} @ {data['Lab'][0]}",
                      'xaxis.title.text': f"Sequence Number", 
                      'yaxis.title.text': f"{mineral[:2]}"
                      }],
            'label': mineral, 
            'method': 'update'
        })

        dropdown_buttons[i]['args'][0]['visible'][i * 5:i * 5 + 5] = [True, True, True, True, True]

    plot_stats_sequence_fig.update_layout(
            updatemenus = [{
                'buttons':  dropdown_buttons,
                'direction': 'down', 
                'showactive': True, 
                'pad': {'r': 5}
            }], 
            title=f"Evelyn Project {data['Original QC'][0]} {data['Standard'][0]} @ {data['Lab'][0]}",
            xaxis_title=f"Sequence Number", 
            yaxis_title=f"{minerals[0][:2]}",
            width=900,  # Adjust this value based on your needs
            height=600  # Adjust this value based on your needs
    )

    plot_stats_sequence_fig.show()






In [ ]:
plot_stats_sequence(stat_data)


In [ ]:
data['Cert Date'] = pd.to_datetime(data['Cert Date'], format='mixed', dayfirst=True)

In [ ]:


def plot_stats_dates(elements):
    plot_stats_dates_fig = go.Figure()
    x_data = data['Cert Date']

    # Add traces for each element
    for element in elements:
        y_data_mean = [stats_df['mean'][element]] * len(x_data) if isinstance(stats_df['mean'][element], (np.float64, float)) else stats_df['mean'][element]
        y_data_analysis = data[element]
        y_data_plus_10 = [stats_df['mean + 10%'][element]] * len(x_data) if isinstance(stats_df['mean + 10%'][element], (np.float64, float)) else stats_df['mean + 10%'][element]
        y_data_minus_10 = [stats_df['mean - 10%'][element]] * len(x_data) if isinstance(stats_df['mean - 10%'][element], (np.float64, float)) else stats_df['mean - 10%'][element]
        y_data_plus_2SD = [stats_df['mean + 2SD'][element]] * len(x_data) if isinstance(stats_df['mean + 2SD'][element], (np.float64, float)) else stats_df['mean + 2SD'][element]
        y_data_minus_2SD = [stats_df['mean - 2SD'][element]] * len(x_data) if isinstance(stats_df['mean - 2SD'][element], (np.float64, float)) else stats_df['mean - 2SD'][element]
        y_data_plus_3SD = [stats_df['mean + 3SD'][element]] * len(x_data) if isinstance(stats_df['mean + 3SD'][element], (np.float64, float)) else stats_df['mean + 3SD'][element]
        y_data_minus_3SD = [stats_df['mean - 3SD'][element]] * len(x_data) if isinstance(stats_df['mean - 3SD'][element], (np.float64, float)) else stats_df['mean - 3SD'][element]

        # Add 8 traces for the current element
        plot_stats_dates_fig.add_trace(go.Scatter(x=x_data, y=y_data_mean, mode='lines', name=f'{element} mean', visible=False))
        plot_stats_dates_fig.add_trace(go.Scatter(x=x_data, y=y_data_analysis, mode='markers', name=f'{element} Analysis', visible=False, marker=dict(color='red')))
        plot_stats_dates_fig.add_trace(go.Scatter(x=x_data, y=y_data_plus_10, mode='lines', name=f'{element} Mean + 10%', visible=False, line=dict(dash='dash', color='red')))
        plot_stats_dates_fig.add_trace(go.Scatter(x=x_data, y=y_data_minus_10, mode='lines', name=f'{element} Mean - 10%', visible=False, line=dict(dash='dash', color='red')))
        plot_stats_dates_fig.add_trace(go.Scatter(x=x_data, y=y_data_plus_2SD, mode='lines', name=f'{element} Mean + 2SD', visible=False, line=dict(dash='dash', color='blue')))
        plot_stats_dates_fig.add_trace(go.Scatter(x=x_data, y=y_data_minus_2SD, mode='lines', name=f'{element} Mean - 2SD', visible=False, line=dict(dash='dash', color='blue')))
        plot_stats_dates_fig.add_trace(go.Scatter(x=x_data, y=y_data_plus_3SD, mode='lines', name=f'{element} Mean + 3SD', visible=False, line=dict(dash='dash', color='green')))
        plot_stats_dates_fig.add_trace(go.Scatter(x=x_data, y=y_data_minus_3SD, mode='lines', name=f'{element} Mean - 3SD', visible=False, line=dict(dash='dash', color='green')))

    # Make the first set of 8 traces visible by default
    for i in range(8):
        plot_stats_dates_fig.data[i].visible = True

    # Create dropdown buttons
    dropdown_buttons = []

    for i, element in enumerate(elements):
        dropdown_buttons.append({
            'args': [{'visible': [False] * len(plot_stats_dates_fig.data)},
                     {'title': f"Evelyn Project {data['Original QC'][0]} {data['Standard'][0]} @ {data['Lab'][0]}",
                      'xaxis.title.text': "Date", 
                      'yaxis.title.text': f"{element[:2]}"}
                    ],
            'label': element, 
            'method': 'update'
        })

        # Set the corresponding 8 traces for the current element to visible
        dropdown_buttons[i]['args'][0]['visible'][i * 8:i * 8 + 8] = [True] * 8

    # Add the dropdown menu and update layout
    plot_stats_dates_fig.update_layout(
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': 'down', 
            'showactive': True, 
            'pad': {'r': 5}
        }],
        title=f"Evelyn Project {data['Original QC'][0]} {data['Standard'][0]} @ {data['Lab'][0]}",
        xaxis_title="Date", 
        yaxis_title=elements[0][:2],  # Default to the first element
        xaxis_rangeslider=dict(visible=True),  # Adding range slider for the dates 
        width=900,  # Adjust this value based on your needs
        height=600  # Adjust this value based on your needs
    )   

    plot_stats_dates_fig.show()

   

    



In [ ]:
plot_stats_dates(stat_data)


In [ ]:
from IPython.display import display, HTML

# HTML and JS for the print button with print styles
html_code = """
    <style>
        @media print {
            @page {
                size: landscape; /* Set the page size to landscape */
            }
            body {
                transform: scale(0.9); /* Optional: Scale down the body if necessary */
                transform-origin: top left; /* Set the transform origin */
            }
        }
    </style>
    <button style="font-size:16px; padding:10px; color:white; background-color:green; border:none; border-radius:5px;" onclick="window.print()">Print this page</button>
"""

# Display the print button in the notebook, which will be included in the HTML output
display(HTML(html_code))